In [1]:
!pip install transformers datasets tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.2 MB/s 
     |████████████████████████████████| 362 kB 50.9 MB/s 
     |████████████████████████████████| 6.6 MB 39.6 MB/s 
     |████████████████████████████████| 596 kB 60.6 MB/s 
     |████████████████████████████████| 101 kB 9.9 MB/s 
     |████████████████████████████████| 212 kB 59.6 MB/s 
     |████████████████████████████████| 1.1 MB 51.6 MB/s 
     |████████████████████████████████| 140 kB 62.3 MB/s 
     |████████████████████████████████| 127 kB 64.0 MB/s 
     |████████████████████████████████| 144 kB 63.2 MB/s 
     |████████████████████████████████| 271 kB 61.0 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing 

A Q&A problem is generally defined as an NLP problem with a given text and a question of AI and getting an answer back. usually, this answer can be found in the original text but there are various approaches to this problem. In the case of visual question answering (VQA), the question is about a visual entity or concept rather than text but the question itself is in the form of text.

Most of the models that are intended to be used inVQA are multimodal models that can understand the visual content along with the question and geneate the answer properly. But, unimodal fully textual QA or just QA is based on textual context and textual questions with respective textual answers.

SQUAD is one of the most well-known datasets in the field of QA. Next we will implement some of the code example of SQUAD and examine them.

In [2]:
from pprint import pprint 
from datasets import load_dataset 
squad = load_dataset("squad") 
for item in squad["train"][1].items(): 
    print(item[0]) 
    pprint(item[1]) 
    print("="*20)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

id
'5733be284776f4190066117f'
title
'University_of_Notre_Dame'
context
('Architecturally, the school has a Catholic character. Atop the Main '
 "Building's gold dome is a golden statue of the Virgin Mary. Immediately in "
 'front of the Main Building and facing it, is a copper statue of Christ with '
 'arms upraised with the legend "Venite Ad Me Omnes". Next to the Main '
 'Building is the Basilica of the Sacred Heart. Immediately behind the '
 'basilica is the Grotto, a Marian place of prayer and reflection. It is a '
 'replica of the grotto at Lourdes, France where the Virgin Mary reputedly '
 'appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive '
 '(and in a direct line that connects through 3 statues and the Gold Dome), is '
 'a simple, modern stone statue of Mary.')
question
'What is in front of the Notre Dame Main Building?'
answers
{'answer_start': [188], 'text': ['a copper statue of Christ']}


However, there is a version 2 of the squad dataset, which has more training smaples, and it is highly recommended to use it. To have an overall understanding of how it is possible to train a model for a QA problem, we will focus on the current part of this problem.

Next we will write some codes to implement the version 2.

In [3]:
squad = load_dataset("squad_v2")
squad

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

As can be seen from the above result there are more than 130000 training samples with more than 11000 validation samples.


As we did before, we must preprocess the data to have the right form to be used by the model. To do so, we must first load our tokenizer which is pretrained tokenizer as long as we are using a pretrained model adn we want to fine-tune it for a QA problem.

In [4]:
from transformers import AutoTokenizer
model = "distilbert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(model)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]